In [1]:
import os
import numpy as np
import cv2

In [2]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [ ]:
def xywh2xyxy(x):
    # convert [x, y, w, h] to [x1, y1, x2, y2]
    y = np.conpy(x)
    y[:, 0] = x[:, 0] - x[:, 2] / 2  # top left x
    y[:, 1] = x[:, 1] - x[:, 3] / 2  # top left y
    y[:, 2] = x[:, 0] + x[:, 2] / 2  # bottom right x
    y[:, 3] = x[:, 1] + x[:, 3] / 2  # bottom right y
    return y

def process(input, mask, anchors):
    anchors = [anchors[i] for i in mask]
    grid_h, grid_w = map(int, input.shape[0:2])
    
    box_confidence = sigmoid(input[..., 4])
    

In [ ]:
if __name__ == "__main__":
    ONNX_MODEL = 'yolov5s.onnx'
    MODEL_NAME = ""
    RKNN_MODEL = f""
    IMG_PATH = "./phone"
    SAVE_PATH = "./save"
    BOX_THRESH=0.2
    NMS_THRESH=0.6
    IMG_SIZE=640  # 目前改为了320
    CLASSES={"person", "phone"}
    masks = []